In [3]:
from hmmlearn import hmm
import numpy as np
import pandas as pd

dimensiune_grid = (9, 10)

culori = [
    "red", "blue", "green", "yellow", 
    "purple", "orange", "pink", "cyan", 
    "brown", "lime"
]

df = pd.read_csv('grid_culori.csv')
grid_culori = df.to_numpy()

dimensiune_grid = grid_culori.shape

observatii = ["red", "red", "lime", "yellow", "blue"]

culoare_to_idx = {culoare: idx for idx, culoare in enumerate(culori)}
observatii_idx = [culoare_to_idx[culoare] for culoare in observatii]

observatii_vector = np.zeros((len(observatii), len(culori)))
for t, idx in enumerate(observatii_idx):
    observatii_vector[t, idx] = 1

n_states = dimensiune_grid[0] * dimensiune_grid[1]
n_observations = len(culori)

transmat = np.zeros((n_states, n_states))

for i in range(dimensiune_grid[0]):
    for j in range(dimensiune_grid[1]):
        idx = i * dimensiune_grid[1] + j
        if j > 0:  
            transmat[idx, idx - 1] = 0.4
        if i > 0:  
            transmat[idx, idx - dimensiune_grid[1]] = 0.15
        if i < dimensiune_grid[0] - 1:  
            transmat[idx, idx + dimensiune_grid[1]] = 0.15
        if j < dimensiune_grid[1] - 1:  
            transmat[idx, idx + 1] = 0.15
        transmat[idx, idx] = 0.15  

transmat = transmat / transmat.sum(axis=1, keepdims=True)

emissionprob = np.zeros((n_states, n_observations))
for i in range(dimensiune_grid[0]):
    for j in range(dimensiune_grid[1]):
        idx = i * dimensiune_grid[1] + j
        culoare = grid_culori[i, j]
        emissionprob[idx, culoare_to_idx[culoare]] = 1.0

model = hmm.MultinomialHMM(n_components=n_states)
model.startprob_ = np.full(n_states, 1.0 / n_states)
model.transmat_ = transmat
model.emissionprob_ = emissionprob

model.n_trials = 1

logprob, state_sequence = model.decode(observatii_vector, algorithm="viterbi")

print("Cea mai probabila secventa de stari:", state_sequence)
print("Probabilitatea secventei:", np.exp(logprob))

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Cea mai probabila secventa de stari: [60 60 61 62 72]
Probabilitatea secventei: 1.5624999999999997e-05


b) Dupa ce am eliminat toate tranzitiile spre stanga, agentul nu mai poate face aceasta miscare iar posibilul traseu depinde de toate celelalte directii sau ramanerea pe loc.

Daca in secventa de observatii agentul s-ar fi mutata spre stanga conf celei mai probabile secvente de stari, secventa noua exclude aceasta optiune, deci vom avea o alta ruta posibila. Probabilitatea totala a noii secvente poate fi mai mica deoarece avem niste constrangeri, reducand numarul de trasee valide.

Pentru analiza, putem compara state_sequence_no_left si state_sequence. Aici se vede cum s-a modificat traseil.